In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest
from sklearn.cluster import DBSCAN
import umap
import hdbscan

d:\Apps\anaconda3\envs\poop\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Wczytanie danych

In [2]:
hc_training = pd.read_csv('../../../classifier_data/HC_training.csv', sep='\t').T
hc_test = pd.read_csv('../../../classifier_data/HC_test.csv', sep='\t').T
oc_training = pd.read_csv('../../../classifier_data/OC_training.csv', sep='\t').T
oc_test = pd.read_csv('../../../classifier_data/OC_test.csv', sep='\t').T

synthetic_hc_data_path = '../../../best_models/HC/WGANGP/25/synthetic_data/'
model_id = os.listdir(synthetic_hc_data_path)[0]
hc_synthetic = pd.read_csv(f'{synthetic_hc_data_path}/{model_id}/generated_data.tsv', sep='\t')

synthetic_oc_data_path = '../../../best_models/OC/WGANGP/25/synthetic_data/'
model_id = os.listdir(synthetic_oc_data_path)[0]
oc_synthetic = pd.read_csv(f'{synthetic_oc_data_path}/{model_id}/generated_data.tsv', sep='\t')

# combine training and test data with labels
hc_training['label'] = 'HC'
hc_test['label'] = 'HC'
oc_training['label'] = 'OC'
oc_test['label'] = 'OC'

hc_synthetic['label'] = 'HC'
oc_synthetic['label'] = 'OC'

training_data = pd.concat([hc_training, oc_training])
test_data = pd.concat([hc_test, oc_test])

In [3]:
# keep only the columns that are in both datasets
# Znalezienie wspólnych kolumn
common_columns = training_data.columns.intersection(test_data.columns)
common_columns = common_columns.intersection(hc_synthetic.columns)
common_columns = common_columns.intersection(oc_synthetic.columns)

# Uporządkowanie kolumn w real_data
training_data = training_data[common_columns]

# Uporządkowanie kolumn w synthetic_data
test_data = test_data[common_columns]

# Uporządkowanie kolumn w synthetic_data
hc_synthetic = hc_synthetic[common_columns]
oc_synthetic = oc_synthetic[common_columns]

common_columns

Index(['ENSG00000081237', 'ENSG00000085265', 'ENSG00000090382',
       'ENSG00000110719', 'ENSG00000115523', 'ENSG00000119535',
       'ENSG00000137642', 'ENSG00000160255', 'ENSG00000177359',
       'ENSG00000198336', 'ENSG00000240356', 'ENSG00000244734',
       'ENSG00000257207', 'label'],
      dtype='object')

In [4]:
# add synthetic data to training data
training_data_synthetic = pd.concat([training_data, hc_synthetic, oc_synthetic])
training_data_synthetic

,ENSG00000081237,ENSG00000085265,ENSG00000090382,ENSG00000110719,ENSG00000115523,ENSG00000119535,ENSG00000137642,ENSG00000160255,ENSG00000177359,ENSG00000198336,ENSG00000240356,ENSG00000244734,ENSG00000257207,label
VUMC-HC-0033-TR2591,6.121758,4.078496,4.643048,4.169415,5.849773,4.399111,4.643048,4.796726,3.083092,7.670810,3.535280,6.995484,9.847484,HC
Vumc-HD-70-TR1062,9.448584,4.926369,5.721557,5.309622,7.717082,4.219669,6.106566,7.143903,5.079361,7.237932,3.701701,10.021953,8.262227,HC
VUMC-HC0053-DOT-HD-48h-TR3087,7.317863,4.845811,4.997802,5.151550,5.593316,3.083092,4.559495,5.935819,3.083092,3.720811,7.783065,7.050183,8.831693,HC
Vumc-HD-149-TR932,5.552469,4.492627,4.275620,3.544794,5.451417,3.083092,3.083092,4.753784,3.544794,3.083092,9.503594,8.661739,3.647372,HC
Vumc-HD-36-TR1165,9.002749,6.525333,5.811822,7.579274,7.306126,5.743631,5.998264,6.846139,4.172174,7.438673,5.671874,9.877834,3.862107,HC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,7.575494,5.317589,5.693978,5.300495,6.821552,4.343516,5.191564,5.940645,6.801260,4.089898,8.197504,8.981106,4.453900,OC
77,4.598385,3.364177,4.205793,3.577808,5.184180,3.360692,3.464017,4.111751,5.542891,3.143812,4.978974,6.425906,6.811332,OC
78,6.376885,6.650597,6.952433,6.044937,6.066106,5.946648,5.110598,6.656283,6.847767,3.205527,6.432171,10.099820,7.040733,OC
79,8.735070,5.471190,4.434652,8.116158,8.708853,5.222122,7.006699,7.191644,4.527420,4.863417,9.014271,9.383606,4.280485,OC


### Wykorzystanie klasyfikatorów

In [5]:
# klasyfikacja z uyciem svm
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score


def svm_classification(training_data, test_data):
    # Oddziel etykiety od cech
    X_train = training_data.drop('label', axis=1)
    y_train = training_data['label']
    X_test = test_data.drop('label', axis=1)
    y_test = test_data['label']

    # Normalizacja danych
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Klasyfikacja przy użyciu SVM
    svm = SVC(kernel='linear', random_state=42)
    svm.fit(X_train, y_train)
    y_pred = svm.predict(X_test)

    # Wypisanie dokładności i innych metryk
    print(f"Dokładność: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))


svm_classification(training_data, test_data)
svm_classification(training_data_synthetic, test_data)

Dokładność: 0.7513513513513513
              precision    recall  f1-score   support

          HC       0.79      0.89      0.84       131
          OC       0.61      0.41      0.49        54

    accuracy                           0.75       185
   macro avg       0.70      0.65      0.66       185
weighted avg       0.73      0.75      0.73       185

[[117  14]
 [ 32  22]]
Dokładność: 0.745945945945946
              precision    recall  f1-score   support

          HC       0.78      0.89      0.83       131
          OC       0.60      0.39      0.47        54

    accuracy                           0.75       185
   macro avg       0.69      0.64      0.65       185
weighted avg       0.73      0.75      0.73       185

[[117  14]
 [ 33  21]]


In [6]:
from sklearn.tree import DecisionTreeClassifier
# klasyfikacja z uyciem random forest
def decision_tree_classification(training_data, test_data):
    # Oddziel etykiety od cech
    X_train = training_data.drop('label', axis=1)
    y_train = training_data['label']
    X_test = test_data.drop('label', axis=1)
    y_test = test_data['label']

    # Normalizacja danych
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Klasyfikacja
    dt = DecisionTreeClassifier(max_depth=5, random_state=42)
    dt.fit(X_train, y_train)
    y_pred = dt.predict(X_test)

    # Wypisanie dokładności i innych metryk
    print(f"Dokładność: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

decision_tree_classification(training_data, test_data)
decision_tree_classification(training_data_synthetic, test_data)

Dokładność: 0.7027027027027027
              precision    recall  f1-score   support

          HC       0.81      0.76      0.78       131
          OC       0.49      0.56      0.52        54

    accuracy                           0.70       185
   macro avg       0.65      0.66      0.65       185
weighted avg       0.71      0.70      0.71       185

[[100  31]
 [ 24  30]]
Dokładność: 0.6648648648648648
              precision    recall  f1-score   support

          HC       0.76      0.77      0.77       131
          OC       0.42      0.41      0.42        54

    accuracy                           0.66       185
   macro avg       0.59      0.59      0.59       185
weighted avg       0.66      0.66      0.66       185

[[101  30]
 [ 32  22]]


In [7]:
from sklearn.ensemble import RandomForestClassifier
# klasyfikacja z uyciem random forest
def random_forest_classification(training_data, test_data):
    # Oddziel etykiety od cech
    X_train = training_data.drop('label', axis=1)
    y_train = training_data['label']
    X_test = test_data.drop('label', axis=1)
    y_test = test_data['label']

    # Normalizacja danych
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Klasyfikacja
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)

    # Wypisanie dokładności i innych metryk
    print(f"Dokładność: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

random_forest_classification(training_data, test_data)
random_forest_classification(training_data_synthetic, test_data)

Dokładność: 0.7621621621621621
              precision    recall  f1-score   support

          HC       0.76      0.96      0.85       131
          OC       0.75      0.28      0.41        54

    accuracy                           0.76       185
   macro avg       0.76      0.62      0.63       185
weighted avg       0.76      0.76      0.72       185

[[126   5]
 [ 39  15]]
Dokładność: 0.7405405405405405
              precision    recall  f1-score   support

          HC       0.76      0.92      0.83       131
          OC       0.61      0.31      0.41        54

    accuracy                           0.74       185
   macro avg       0.69      0.62      0.62       185
weighted avg       0.72      0.74      0.71       185

[[120  11]
 [ 37  17]]


In [8]:
from xgboost import XGBClassifier

# klasyfikacja z uyciem xgboost
def xgboost_classification(training_data, test_data):
    # Oddziel etykiety od cech
    X_train = training_data.drop('label', axis=1)
    y_train = training_data['label']
    X_test = test_data.drop('label', axis=1)
    y_test = test_data['label']

    # Normalizacja danych
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Klasyfikacja
    y_train = y_train.replace('HC', 0)
    y_train = y_train.replace('OC', 1)
    y_test = y_test.replace('HC', 0)
    y_test = y_test.replace('OC', 1)
    xgb = XGBClassifier(random_state=42)
    xgb.fit(X_train, y_train)
    y_pred = xgb.predict(X_test)

    # Wypisanie dokładności i innych metryk
    print(f"Dokładność: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

xgboost_classification(training_data, test_data)
xgboost_classification(training_data_synthetic, test_data)

Dokładność: 0.7675675675675676
              precision    recall  f1-score   support

           0       0.80      0.89      0.84       131
           1       0.64      0.46      0.54        54

    accuracy                           0.77       185
   macro avg       0.72      0.68      0.69       185
weighted avg       0.75      0.77      0.76       185

[[117  14]
 [ 29  25]]
Dokładność: 0.7351351351351352
              precision    recall  f1-score   support

           0       0.79      0.85      0.82       131
           1       0.56      0.46      0.51        54

    accuracy                           0.74       185
   macro avg       0.67      0.66      0.66       185
weighted avg       0.72      0.74      0.73       185

[[111  20]
 [ 29  25]]


In [9]:
from sklearn.neural_network import MLPClassifier

# klasyfikacja z uzyciem sieci neuronowych

def nn_classification(training_data, test_data):
    # Oddziel etykiety od cech
    X_train = training_data.drop('label', axis=1)
    y_train = training_data['label']
    X_test = test_data.drop('label', axis=1)
    y_test = test_data['label']

    # Normalizacja danych
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Klasyfikacja
    nn = MLPClassifier(hidden_layer_sizes=(100,100), random_state=42)
    nn.fit(X_train, y_train)
    y_pred = nn.predict(X_test)

    # Wypisanie dokładności i innych metryk
    print(f"Dokładność: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

nn_classification(training_data, test_data)
nn_classification(training_data_synthetic, test_data)

d:\Apps\anaconda3\envs\poop\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Dokładność: 0.7621621621621621
              precision    recall  f1-score   support

          HC       0.84      0.82      0.83       131
          OC       0.59      0.63      0.61        54

    accuracy                           0.76       185
   macro avg       0.71      0.72      0.72       185
weighted avg       0.77      0.76      0.76       185

[[107  24]
 [ 20  34]]
Dokładność: 0.7351351351351352
              precision    recall  f1-score   support

          HC       0.82      0.80      0.81       131
          OC       0.54      0.57      0.56        54

    accuracy                           0.74       185
   macro avg       0.68      0.69      0.68       185
weighted avg       0.74      0.74      0.74       185

[[105  26]
 [ 23  31]]


d:\Apps\anaconda3\envs\poop\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
